<a href="https://colab.research.google.com/github/ShoAnn/Legal_Document_IR/blob/main/TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os, glob, re, sys, random, unicodedata, collections
import zipfile
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

from tqdm import tqdm
from functools import reduce
from collections import Counter


STOP_WORDS = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def read_and_combine_xml_from_zip(zip_file_path, n):
    """
    Reads the first n XML files from a ZIP file, concatenates them into a single DataFrame,
    and returns the combined DataFrame.

    Args:
        zip_file_path (str): The path to the ZIP file.

    Returns:
        pd.DataFrame: The combined DataFrame from the parsed XML files.
    """

    data = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        file_count = 0
        for filename in zip_ref.namelist():
            if filename.endswith('.xml'):
                try:
                    with zip_ref.open(filename) as xml_file:
                        # Use the appropriate XPath expression based on your needs
                        rows = pd.read_xml(xml_file, xpath='//putusan')  # Example: Read all "putusan" elements
                        data.append(rows)
                        file_count += 1
                        if file_count >= n:
                            break
                except Exception as e:
                    print(f"Error parsing XML file '{filename}': {e}")

    # Concatenate DataFrames into a single DataFrame
    combined_df = pd.concat(data, ignore_index=True)
    return combined_df

In [5]:
zip_file_path = '/content/drive/MyDrive/Legal_IR/Dataset/indo-law-main.zip'  # Replace with your ZIP file path
df = read_and_combine_xml_from_zip(zip_file_path, 1001)

In [6]:
df

,amar,amar_lainnya,id,klasifikasi,lama_hukuman,lembaga_peradilan,provinsi,status,sub_klasifikasi,url,...,identitas,riwayat_penahanan,riwayat_perkara,riwayat_tuntutan,riwayat_dakwaan,fakta,amar_putusan,penutup,fakta_hukum,pertimbangan_hukum
0,pidana,hukum,00035681c8d944203f25d2e8215ae2bf,pidana-umum,210,pn-kudus,jateng,berkekuatan-hukum-tetap,pemalsuan,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap eny sulistiyaningsih binti mashad...,terdakwa ditahan dengan jenis tahanan rutan se...,pengadilan negeri tersebut\nsetelah membaca be...,setelah mendengar tuntutan requsitoir penuntut...,menimbang bahwa terdakwa diajukan di persidang...,menimbang bahwa selanjutnya untuk membuktikan ...,mengadili 1 menyatakan terdakwa eny sulistiyan...,demikian diputuskan dalam rapat permusyawarata...,NaN,NaN
1,pidana,hukum,000399ce26773e18695ce14f519cb9e6,pidana-umum,720,pn-demak,jateng,berkekuatan-hukum-tetap,pencurian,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ali maftuhin bin nur salim tempat...,terdakwa ditahan di rumah tahanan negara berda...,pengadilan negeri tersebut\nsetelah membaca\np...,setelah mendengar surat tuntutan pidana requis...,menimbang bahwa terdakwa didakwa oleh penuntut...,menimbang bahwa untuk menguatkan dakwaan terse...,mengadili 1 menyatakan terdakwa ali maftuhin b...,demikianlah diputuskan dalam rapat permusyawar...,menimbang bahwa berdasarkan keterangan saksi s...,NaN
2,pidana,jatuh-pidana-oleh-karena-itu-kepada-dakwa-ir-b...,0006582ad67cd9bd1ddf4261a09bf382,pidana-umum,120,pn-kediri,jatim,berkekuatan-hukum-tetap,kejahatan-terhadap-keamanan-negara,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ir bambang sasongko bin r soewarn...,terdakwa tidak ditahan,terdakwa didampingi oleh penasehat hukumnya ya...,telah mendengar pembacaan tuntutan pidana oleh...,menimbang bahwa terdakwa diajukan di persidang...,menimbang bahwa selanjutnya dipersidangan tela...,mengadili\n1 menyatakan terdakwa ir bambang sa...,demikian diputuskan dalam rapat musyawarah maj...,NaN,NaN
3,pidana,hukum,00092bbac1a705aa44f2e10a0511cc0c,pidana-khusus,3240,pn-klaten,jateng,berkekuatan-hukum-tetap,anak,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap jk alias eklek tempat lahir klate...,penyidik sejak tanggal 14 januari 2013 sampai ...,terdakwa dalam perkara ini didampingi penaseha...,setelah mendengar pembacaan tuntutan pidana da...,menimbang bahwa terdakwa diajukan di persidang...,menimbang bahwa karena terdakwa menyangkal mel...,mengadili\n1 menyatakan terdakwa jk alias ekle...,demikianlah diputuskan dalam rapat permusyawar...,menimbang bahwa berdasarkan keterangan saksi s...,NaN
4,pidana,hukum,0009b7fa2e45129b1755ddbdf35c7eec,pidana-khusus,300,pn-blora,jateng,berkekuatan-hukum-tetap,tidak-diketahui,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap oktavia cokrodiharjo bin poly cok...,terdakwa tidak ditahan,terdakwa dalam persidangan didampingi oleh pen...,setelah mendengar pembacaan tuntutan pidana ya...,menimbang bahwa terdakwa diajukan ke persidang...,menimbang bahwa untuk membuktikan dakwaannya p...,mengadili\n1 menyatakan terdakwa oktavia cokro...,demikian diputuskan dalam sidang permusyawarat...,menimbang bahwa berdasarkan alat bukti dan bar...,menimbang selanjutnya bahwa majelis hakim akan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,pidana,pidana-penjara-lama-4-empat-tahun-dan-8-delapa...,0a884bfac05948c4e5fa8d041a7554de,pidana-khusus,1680,pn-yogyakarta,yogya,berkekuatan-hukum-tetap,narkotika-dan-psikotropika,https://putusan3.mahkamahagung.go.id/direktori...,...,NaN,NaN,telah mendengar keterangan para saksi dan terd...,telah melihat dan meneliti barang bukti yang d...,menimbang bahwa terdakwa diajukan oleh penuntu...,menimbang bahwa di persidangan telah di dengar...,mengadili\n1 menyatakan bahwa terdakwa wib wic...,demikianlah diputuskan dalam rapat permusyawar...,NaN,NaN
997,pidana,pidana-penjara-waktu-

In [7]:
df_meta = df[['amar', 'amar_lainnya', 'id', 'klasifikasi', 'lama_hukuman',
       'lembaga_peradilan', 'provinsi', 'status', 'sub_klasifikasi', 'url']]

df_content = pd.DataFrame()
df_content['text'] = df[['kepala_putusan', 'identitas', 'riwayat_penahanan', 'riwayat_perkara',
       'riwayat_tuntutan', 'riwayat_dakwaan', 'fakta', 'amar_putusan',
       'penutup', 'fakta_hukum', 'pertimbangan_hukum']].agg(lambda x: '\n'.join(x.dropna()), axis=1)
  # Joins with \n as the separator

df_content

,text
0,putusan\nno 45 pid b 2014 pn kds\ndemi keadila...
1,putusan\nnomor 157 pid b 2017 pn dm\ndemi kead...
2,putusan\nn o 03 pid s 2011 pn kdr\ndemi keadil...
3,putusan\nnomor 22 pid sus 2013 pn kl\ndemi kea...
4,putusan\nnomor 4 pid sus 2015 pn bl\ndemi kead...
...,...
996,putusan\nnomor 69 pid b 2011 pn yk\ndemi keadi...
997,putusan\nnomor 321 pid b 2018 pn bkl\ndemi kea...
998,putusan\nnomor 262 pid b 2016 pn bk\ndemi kead...
999,putusan\nnomor 479 pid sus 2017 pn bl\ndemi ke...


# Preprocessing

In [8]:
def split_strings(row):
    return row.str.split('\n')

WORD_MIN_LENGTH = 2
def tokenize_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in STOP_WORDS and len(word) >= WORD_MIN_LENGTH]
    return words

def inverted_index(words):
    """Create a inverted index of words (tokens or terms) from a list of terms

    Parameters:
    words (list of str): tokenized document text

    Returns:
    Inverted index of document (dict)

   """
    inverted = {}
    for index, word in enumerate(words):
        locations = inverted.setdefault(word, [])
        locations.append(index)
    return inverted

def inverted_index_add(inverted, doc_id, doc_index):
    """Insert document id into Inverted Index

    Parameters:
    inverted (dict): Inverted Index
    doc_id (int): Id of document been added
    doc_index (dict): Inverted Index of a specific document.

    Returns:
    Inverted index of document (dict)

   """
    for word in doc_index.keys():
        locations = doc_index[word]
        indices = inverted.setdefault(word, {})
        indices[doc_id] = locations
    return inverted

In [9]:
# segment into sentences
df_content = df_content.apply(split_strings, axis=1)
df_content

,text
0,"[putusan, no 45 pid b 2014 pn kds, demi keadil..."
1,"[putusan, nomor 157 pid b 2017 pn dm, demi kea..."
2,"[putusan, n o 03 pid s 2011 pn kdr, demi keadi..."
3,"[putusan, nomor 22 pid sus 2013 pn kl, demi ke..."
4,"[putusan, nomor 4 pid sus 2015 pn bl, demi kea..."
...,...
996,"[putusan, nomor 69 pid b 2011 pn yk, demi kead..."
997,"[putusan, nomor 321 pid b 2018 pn bkl, demi ke..."
998,"[putusan, nomor 262 pid b 2016 pn bk, demi kea..."
999,"[putusan, nomor 479 pid sus 2017 pn bl, demi k..."


In [10]:
# Build indexes
inverted_doc_indexes = {}
files_with_index = []
files_with_tokens = {}
doc_id=0

for i, row in tqdm(df_content.iterrows(), total=len(df_content)):
    for text in row['text']:
        #Clean and Tokenize text of each document
        words = tokenize_text(text)
        #Store tokens
        files_with_tokens[doc_id] = words

        doc_index = inverted_index(words)
        inverted_index_add(inverted_doc_indexes, doc_id, doc_index)
        files_with_index.append(f"doc-{i+1}")
        doc_id = doc_id+1

100%|██████████| 1001/1001 [01:17<00:00, 12.84it/s]


In [11]:
count = 0
for key, value in inverted_doc_indexes.items():
    print(f"{key}: {value}")
    count += 1
    if count == 5:  # Print only the first 5 items
        break

putusan: {0: [0], 3: [12], 31: [9], 32: [11], 226: [8, 14], 321: [2], 326: [19], 340: [28], 344: [0], 347: [10], 487: [16], 490: [3], 725: [17], 727: [3, 15], 752: [22], 853: [21], 861: [29], 875: [0], 878: [13], 1772: [17], 1784: [17], 1789: [4], 1801: [27], 1804: [0], 1807: [12], 2030: [27], 2051: [2, 9, 11], 2121: [13], 2123: [0], 2126: [10], 2159: [11], 2381: [12], 2501: [5], 2504: [18], 2505: [4, 25, 40, 47], 2534: [0], 2537: [12], 2656: [29], 2730: [25], 2731: [23], 2733: [0], 2736: [10], 2873: [12], 2881: [4], 2891: [0], 2894: [15], 3093: [3, 11], 3096: [6], 3101: [22], 3208: [18], 3222: [19], 3224: [0], 3227: [10], 3553: [0], 3556: [12], 3570: [11], 3891: [25], 3928: [0], 3931: [10], 3959: [9], 3979: [12], 4003: [23], 4005: [0], 4008: [11], 4171: [23], 4191: [22], 4192: [16], 4194: [2], 4206: [0], 4209: [12], 4379: [0], 4383: [10], 4401: [7], 4569: [0], 4572: [13], 4689: [3, 13], 4701: [26], 4723: [11], 4774: [24], 4777: [20], 4783: [2], 4784: [14], 4788: [4], 4794: [5], 4796: 

# TF-IDF

In [12]:
## number of documents each term occurs
DF = {}
for word in inverted_doc_indexes.keys():
    DF[word] = len([doc for doc in inverted_doc_indexes[word]])

total_vocab_size = len(DF)
print(total_vocab_size)

55943


In [13]:
tf_idf = {} # Our data structure to store Tf-Idf weights

N = len(files_with_tokens)

for doc_id, tokens in tqdm(files_with_tokens.items()):

    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):

        # Calculate Tf
        tf = counter[token] # Counter returns a tuple with each terms counts
        tf = 1+np.log(tf)

        # Calculate Idf
        if token in DF:
            df = DF[token]
        else:
            df = 0
        idf = np.log((N+1)/(df+1))

        # Calculate Tf-idf
        tf_idf[doc_id, token] = tf*idf

100%|██████████| 250408/250408 [00:32<00:00, 7788.13it/s] 


In [14]:
def ranked_search(k, tf_idf_index, file_names, query):
    """Run ranked query search using tf-idf model.

    Parameters:
    k (int): number of results to return
    tf_idf_index (dict): Data Structure storing Tf-Idf weights to each
                        pair of (term,doc_id)
    file_names (list): List with names of doc
    query (txt): Query text

    Returns:
    Top-k document that matchs the query.

   """
    tokens = tokenize_text(query)
    query_weights = {}
    for doc_id, token in tf_idf:
        if token in tokens:
            query_weights[doc_id] = query_weights.get(doc_id, 0) + tf_idf_index[doc_id, token]

    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)
    results = []
    print(query_weights[:k])
    for i in query_weights[:k]:
        results.append(file_names[i[0]])

    return results

In [15]:
print(ranked_search(4, tf_idf, files_with_index,
                    "yayat"))

[(250129, 16.971572935624938), (231638, 13.692558182749336), (231643, 13.692558182749336), (231772, 13.692558182749336)]
['doc-1000', 'doc-929', 'doc-929', 'doc-929']
